In [1]:
# imports
import re
import requests
import pandas as pd
from scrapy import Selector

In [2]:
# get first aircraft type info
request = requests.get('https://www.saudia.com/experience/about-us/our-fleet/airbus-a320-214')

In [3]:
request.status_code

200

In [4]:
# get html text
html = request.text

In [5]:
sel = Selector(text=html)

In [6]:
# extract aircraft names
titles = sel.xpath('//h3[@class="fs-1-3 vg-0--top"]')[::3].xpath('.//text()').extract()
titles

['AIRBUS A320CEO - (32N)',
 'AIRBUS A320-214 (STANDARD CONFIGURATION) – (320)',
 'AIRBUS A320-214 (BUSINESS CONFIGURATION) – (32E)',
 'AIRBUS A320CEO - (RETROFIT CONFIGURATION) – (32U)']

In [7]:
# extract tables of interest (aircraft specs & seat specs)
tables = sel.xpath('//table[@class="full styled"]')

In [8]:
# split tables by aircraft
aircraft_tables = {}
for i in range(len(titles)):
    aircraft_tables[titles[i]] = tables[i*3:(i*3)+3]

In [9]:
aircraft_data = {}
for title in titles:
    data = {}
    tables = aircraft_tables[title]
    for table in tables:
        table_title = table.xpath('./caption/text()').extract_first()
        if "Aircraft Specifications" in table_title:
            headers = table.xpath('.//th/text()').extract()
            datapoints = table.xpath('.//td/text()').extract()
            print(len(headers) == len(datapoints))
            for i in range(len(headers)):
                data[headers[i]] = datapoints[i].strip()
        else:
            headers1 = table.xpath('.//th//text()').extract()[1:]
            headers2 = []
            datapoints = []
            rows = table.xpath('./tbody//tr')
            for row in rows:
                longheader = row.xpath('.//td//text()').extract_first()
                headers2.append(longheader)
                if longheader != 'Amenities':
                    datapoints.append(row.xpath('.//td//text()')[1:].extract())
                else:
                    datapoints.append(row.xpath('.//td//text()')[1:].extract())
            for i in range(len(headers1)):
                for j in range(len(headers2)):
                    data[headers1[i]+" "+headers2[j]] = datapoints[j][i].strip()
                
            print(len(headers2) == len(datapoints))
            print(headers1)
            print(headers2)
            print(datapoints)
    aircraft_data[title] = data

True
True
True
['Economy Class', 'Business Class']
['Accommodation (144 seats)', 'Seat width/Pitch', 'ABREAST', 'Amenities']
[['132 seats', '12 seats'], ['16.3 in - 18 in/ 30 in - 31 in', '21 in/ 45 in'], ['3-3', '2-2'], ['\xa0', '\xa0', '\xa0', '\xa0', '\xa0', '\xa0', '\xa0', '\xa0', '\xa0', '\xa0']]
True
True
True
['Economy Class', 'Business Class']
['Accommodation (132 seats)', 'Seat width/Pitch', 'ABREAST', 'Amenities']
[['120 seats', '12 seats'], ['17.17 in -18.33 in / 32 in -33 in', '21 in / 45 in'], ['3-3', '2-2'], ['\xa0', '\xa0', '\xa0', '\xa0']]
True
True
True
['Economy Class', 'Business Class']
['Accommodation (116 seats)', 'Seat width/Pitch', 'ABREAST', 'Amenities']
[['96 seats', '20 seats'], ['17.17 in - 18.33 in / 32 in', '21 in / 54 in'], ['3-3', '2-2'], ['\xa0', '\xa0', '\xa0', '\xa0', '\xa0', '\xa0']]
True
True
True
['Economy Class', 'Business Class']
['Accommodation (110 seats)', 'Seat width/Pitch', 'ABREAST', 'Amenities']
[['90 seats', '20 seats'], ['16.3 in - 18 in/

In [10]:
aircraft_data

{'AIRBUS A320CEO - (32N)': {'Seating capacity': '12 Business Class, 132 Guest Class',
  'length': '123 ft (37.57 m)',
  'Wingspan': '117 ft (35.80 m)',
  'Cruising speed': '447 knots (511 mph, 828 km/h)',
  'Maximum cruising altitude': '39,100-41,000 ft (11.900-12.500 m)',
  'Range': '3,300 nautical miles (6,100 km)',
  'Economy Class Accommodation (144 seats)': '132 seats',
  'Economy Class Seat width/Pitch': '16.3 in - 18 in/ 30 in - 31 in',
  'Economy Class ABREAST': '3-3',
  'Economy Class Amenities': '',
  'Business Class Accommodation (144 seats)': '12 seats',
  'Business Class Seat width/Pitch': '21 in/ 45 in',
  'Business Class ABREAST': '2-2',
  'Business Class Amenities': ''},
 'AIRBUS A320-214 (STANDARD CONFIGURATION) – (320)': {'Seating capacity': '12 Business Class, 120 Guest Class',
  'length': '123 ft (37.57 m)',
  'Wingspan': '112 ft (34.10 m)',
  'Cruising speed': '447 knots (511 mph, 828 km/h)',
  'Maximum cruising altitude': '36,100 ft (11,000 m)',
  'Range': '3.200 

In [11]:
print(tables[2].extract())

<table class="full styled" border="0">
<caption class="accessible-hide">Seat Specifications AIRBUS A320CEO</caption>

<thead>
<tr class="bold header-color-1">
<th style=" border-left: 0px; width: 191px;">Seat Specifications</th>
<th style=" width: 196px;">Economy Class</th>
<th>Business Class</th>
</tr>

</thead><tbody>
<tr>
<td style=" width: 191px;"><strong>Accommodation (110 seats)</strong></td>
<td style=" width: 196px;">90 seats</td>
<td>20 seats</td>
</tr>

<tr>
<td style=" width: 191px;"><strong>Seat width/Pitch</strong></td>
<td style=" width: 196px;">16.3 in - 18 in/ 31 in</td>
<td>20 in/ 63 in</td>
</tr>

<tr>
<td style=" width: 191px;"><strong>ABREAST</strong></td>
<td style=" width: 196px;">3-3</td>
<td>2-2</td>
</tr>

<tr>
<td style=" width: 191px;"><strong>Amenities</strong></td>
<td style=" width: 196px;"><img alt="Wi-Fi" title="Wi-Fi" src="/SV_Content/images/Wi-Fi.png" tabindex="0"> <img alt="IFE Streaming" title="IFE Streaming" src="/SV_Content/images/IFE%20Streaming.p

In [12]:
tables[0].xpath('.//th//text()').extract()

['Seating capacity', 'length', 'Wingspan']

In [13]:
tds = []
for sel in tables[0].xpath('./tbody//tr'):
    tds.append(sel.xpath('.//td//text()'))
tds[0].extract()

['20 Business Class, 90 Guest Class',
 '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0123 ft (37.57 m) \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0',
 '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0112 ft (34.10 m) \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0']